## Загрузка и подготовка данных

Загрузка данных из `zip` архива.

In [ ]:
from zipfile import ZipFile
import json

myzip = ZipFile('hr.zip', 'r')
names_list = myzip.namelist()

names_list

Функция для выгрузке данных из `json-ld` файлов.

In [ ]:
def extract_json(file):
    raw_data = myzip.open(file)
    return json.load(raw_data)
    
extract_json('vc_body_urn_uuid_7e8f96fc-3617-45e5-92a6-a7711c5a7225.jsonld')

Функция для поиска ссылок в данном документе.

In [ ]:
def find_links(json_file):
    subject = json_file['credentialSubject']
    ids = []
    for sub_key in subject.keys():
        if type(subject[sub_key]) == str and (subject[sub_key][:8] == 'urn:uuid' or subject[sub_key][:8] == 'url:uuid'):
            curr_id = subject[sub_key]
            ids.append(curr_id)
        elif type(subject[sub_key]) == str:
            curr_id = subject[sub_key]
            props.append(curr_id)
    return ids

find_links(extract_json('vc_body_urn_uuid_7e8f96fc-3617-45e5-92a6-a7711c5a7225.jsonld'))

Функция для поиска полей данных в данном документе.

In [ ]:
def find_properties(json_file):
    subject = json_file['credentialSubject']
    props = {}
    for sub_key, sub_value in subject.items():
        if type(sub_value) == str and (sub_value[:8] != 'urn:uuid' and sub_value[:8] != 'url:uuid'):
            props[sub_key] = sub_value
    return props

find_properties(extract_json('vc_body_urn_uuid_7e8f96fc-3617-45e5-92a6-a7711c5a7225.jsonld'))

## Загрузка данных в БД

In [384]:
from gremlin_python import statics
from gremlin_python.structure.graph import Graph
from gremlin_python.process.traversal import T
from gremlin_python.process.graph_traversal import __
from gremlin_python.process.strategies import *
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection

graph = Graph()

remoteConn = DriverRemoteConnection('wss://neptunecdkcluster6a0221d7-tf6zid0yicr7.cluster-cnhnmqpcjens.eu-west-1.neptune.amazonaws.com:8182/gremlin','g')
g = graph.traversal().withRemote(remoteConn)

Добавляем все документы как вершины с соответствующими полями данных.

In [390]:
def add_vertex(item):
    if item[0] != 'v' or item[1] != 'c':
        return
    info = extract_json(item)
    props_map = find_properties(info)
    props = list(props_map.items())
    print(info['credentialSubject']['type'])
    g.addV('tulchin-vc').property('id', info['id']).next()
    j = 0
    for key, value in props:
        print(j)
        j += 1
        g.V().has('id', info['id']).hasLabel('tulchin-vc').property(key, value).next()

In [391]:
for item in names_list:
    add_vertex(item)

CompanyAddendum
0
1
2
3
4
5
6
7
CandidateAgreement
0
1
2
CandidateAgreement
0
1
2
UtilityBill
0
1
2
3
4
5
JobOffer
0
1
2
3
4
5
6
7
8
9
KvkExtract
0
1
2
3
4
5
6
7
8
9
ResignationConfirmation
0
1
2
3
4
5
6
EmploymentAgreement
0
1
EmploymentAgreement
0
1
ResignationNotice
0
1
2
3
CompanyAddendum
0
1
2
3
4
5
6
7
JobOffer
0
1
2
3
4
5
6
7
8
9
CompanyAddendum
0
1
2
3
4
5
6
7
Passport
0
1
2
3
4
5
6
7
8
9
10
11
12
13
CompanyAddendum
0
1
2
3
4
5
6
7
Passport
0
1
2
3
4
5
6
7
8
9
10
11
12
13
Passport
0
1
2
3
4
5
6
7
8
9
10
11
12
13
EmploymentAgreement
0
1
ResignationConfirmation
0
1
2
3
4
5
6
UtilityBill
0
1
2
3
4
5
UtilityBill
0
1
2
3
4
5
ResignationNotice
0
1
2
3
JobOffer
0
1
2
3
4
5
6
7
8
9
CandidateAgreement
0
1
2
UtilityBill
0
1
2
3
4
5
JobOffer
0
1
2
3
4
5
6
7
8
9
Passport
0
1
2
3
4
5
6
7
8
9
10
11
12
13
CandidateAgreement
0
1
2
JobOffer
0
1
2
3
4
5
6
7
8
9
UtilityBill
0
1
2
3
4
5
Passport
0
1
2
3
4
5
6
7
8
9
10
11
12
13
Payslip
0
1
2
3
4
EmploymentAgreement
0
1
JobOffer
0
1
2
3
4
5
6
7
8
9


Проведем ребра в нашем графе:

_Строго говоря никто не обещает нам, что вершина в которую мы проводим ребро существует. Для того чтобы отслеживать подобные ситуации желательно поддерживать hashmap адресов всех `VC`, которые были найдены нами в файлах и добавить туда `VC`, упомянутые в связях(хотя, конечно, это совершенно наивное решение и лучше бы такое вообще запрещать)_

In [392]:
def add_edges(item):
    if item[0] != 'v' or item[1] != 'c':
        return
    info = extract_json(item)
    edges_ends = find_links(info)
    current_vertex = g.V().has('id', info['id']).hasLabel('tulchin-vc').toList()[0]
    c = 0
    for neig_end in edges_ends:
        print(c)
        c += 1
        neig_vertex = g.V().has('id', neig_end).hasLabel('tulchin-vc').toList()[0]
        g.V(current_vertex).addE('tulchin-link').to(neig_vertex).next()

In [393]:
for item in names_list:
    add_edges(item)

0
0
1
2
0
1
2
0
1
0
0
0
0
0
0
1
0
0
0
0
0
0
1
0
1
2
0
1
0
1
2
0
1
0
0
1
0
0
1
0
1
2
0
0
1
2
0
0
1
0
0
1
0
0
1
0
0
1
0
1
0
0
1
2
0
1
2
0
0
1
0
0
0
1
2
0
1


## Визуализация данных

In [394]:
visual = '{"tulchin-vc":"type"}'

In [ ]:
%%gremlin -d $visual

g.V().hasLabel('tulchin-vc').outE().path().by(elementMap())

## Оптимальный подход к получению списка сотрудников

In [ ]:
%%gremlin

g.V().hasLabel('tulchin-vc').has('type', 'EmploymentAgreement').as("Agreements")
    .out().has('type', 'CandidateAgreement')
    .out().has('type', 'Passport').as("Surname").as('id').select("Agreements").out()
    .until(__.not(in().in().count().is(0)))
    .repeat(in())
    .where(__.not(has('type', 'ResignationNotice').or().has('type', 'ResignationConfirmation')))
    .where(__.in().count().is(gt(0)))
    .coalesce(
        has('type', 'CandidateAgreement').out().has('type', 'JobOffer').values('OfferedPosition').limit(1),
        has('type', 'EmploymentAgreement').in().order().by('From', desc).values('Position').limit(1),
        has('type', 'CompanyAddendum').values('Position').limit(1),
        has('type', 'EmploymentAddendum').in().order().by('From', desc).values('Position').limit(1)
    ).as('Position')
    .select('Surname', 'Position', 'id').by('Surname').by().by(T.id).dedup()

##  Добавление новых документов

In [397]:
def add_documents(list_of_documents): 
    for item in list_of_documents:
        add_vertex(item)
        add_edges(item)

## Удаление данных из БД

In [ ]:
%%gremlin

g.V().hasLabel('tulchin-vc').drop().iterate()
g.E().hasLabel('tulchin-link').drop().iterate()